## Community Detection

In [ ]:
# !pip install python-louvain
# !pip install networkx
# !pip install python-igraph

In [1]:
%%time
%pylab inline
import networkx as nx
import numpy as np
import matplotlib 
import scipy
import warnings

import json 

import community as community_louvain
import igraph

import operator

from PIL import Image
from numpy import asarray
from numpy import save
from scipy import misc
import glob

import matplotlib.pyplot as plt
from IPython.display import Image

from tqdm import tqdm
import time

import os, os.path

Populating the interactive namespace from numpy and matplotlib
CPU times: user 1.3 s, sys: 1.94 s, total: 3.24 s
Wall time: 1.61 s


#### Code for Unzipping songnet compressed. Run once.

In [ ]:
# import gzip

# #Define the file's location
# file_path = './playlist-songnet-compressed.gz'

# #Open the file and read its contents
# with gzip.open(file_path, "rb") as file:
#     file_content = file.read()


# #Save the new txt file
# txt_file_name = "songnet.txt"

# with open(txt_file_name, "w") as file:
#     file.write(file_content)

# # import gzip
# # with gzip.open('./playlist-songnet-compressed.gz', 'rb') as f, open('songnet.txt', 'w') as f_out:
# #     f_out.write(f.read())

### Load in Validation spectrograms and reduced_songnet

In [12]:
val_path = '/datasets/home/21/321/ee228sp20ta1/G51/val_specs/'
songnet_edgelist ="./reduced_songnet.txt"

In [13]:
songs=nx.read_edgelist(songnet_edgelist,create_using=nx.DiGraph())
node, edge=songs.order(),songs.size()
print("No. of nodes are=",node)
print("No. of edges are=",edge)

No. of nodes are= 4280
No. of edges are= 429918


### Helper function to find largest connected component

In [4]:
def find_largest_component(generator):
    sub_graphs = []
    for item in generator:
        sub_graphs.append(item)

    list_of_all_subgraphs = [(graph, len(graph.nodes)) for graph in sub_graphs]

    largest_count = 0
    for i in range(len(list_of_all_subgraphs)):
        count = list_of_all_subgraphs[i][1]
        if count > largest_count:
            largest_count = count
            largest_component = list_of_all_subgraphs[i][0]
    return largest_component

### Find Largest Connected Component

In [5]:
songs_ud = songs.to_undirected()
songs_ud_components = nx.connected_component_subgraphs(songs_ud)
songs_largest_component = find_largest_component(songs_ud_components)

### Find All Song ID's in the spectrogram validation set

In [6]:
songnet_ids = []
for image_path in tqdm(glob.glob(val_path + "/*.png")):
    
    
    # Load and collect song ids into list of strings
    ID = os.path.basename(image_path).partition(".png")[0]

    if ID in list(songs_largest_component.nodes):
        
        songnet_ids.append(ID)

100%|██████████| 2763/2763 [00:00<00:00, 16915.33it/s]


### Remove nodes from the network that aren't in the validation set

In [7]:
remove_nodes = []

for node in list(songs_largest_component.nodes):
    
    if node not in songnet_ids:
        remove_nodes.append(node)
        
songs_largest_component.remove_nodes_from(remove_nodes)

ids = list(songs_largest_component.nodes)

In [ ]:
# path, dirs, files = next(os.walk(val_path))
# file_count = len(files)
# print(file_count)

### Optional: Run Community_louvain

In [ ]:
#songs_community = community_louvain.best_partition(songs_largest_component)
# songs_sorted = sorted_x = sorted(songs_community.items(), key=operator.itemgetter(1))
# songs_coms = dict(songs_sorted)

#Save out community detection with louvain method
# json = json.dumps(songs_coms)
# f = open("val_communities_1.json","w")
# f.write(json)
# f.close()

### Optional: Compute Node Degrees

In [27]:
songs_ud_degrees = songs_largest_component.degree()
songs_ud_degrees = dict(songs_ud_degrees)

### Save out largest connected component

### This is necessary for loading the network into the igraph module below

In [28]:
# nx.set_node_attributes(songs_largest_component, songs_coms, "community")
nx.set_node_attributes(songs_largest_component, songs_ud_degrees, "degrees")
# # nx.set_node_attributes(songs_largest_component, song_partitions, "partitions")

In [29]:
nx.write_gml(songs_largest_component, "songnet_lc.gml")

### Run igraph community Detection

### First, define number of communities, for example as informed by optimal number of clusters analysis on spectrogram latent vectors in notebook 04

In [30]:
n_communities  = 3

In [36]:
songs_lc = igraph.read('songnet_lc.gml', format="gml")
song_ptns = songs_lc.community_leading_eigenvector(clusters=3)
song_partitions = {}

for cluster in range(len(song_ptns)):
    for node in song_ptns[cluster]:
        idx = ids[node]
        song_partitions[idx] = cluster

In [35]:
# If minimum computed_number_of_communities > optimal number of kmeans clusters,
# will have to make a compromise: (rerun kmeans with k = minimum # of communities)
# Alternatively, Explore Kmeans optimal k versus various # of communities

In [38]:
computed_number_of_communities = max(song_partitions.values())

# This number should equal n_communities
print(computed_number_of_communities + 1)

4


### Save out igraph communities as JSON

In [11]:
# Save out community detection with igraph
json = json.dumps(song_partitions)
f = open("val_communities_3.json","w")
f.write(json)
f.close()

### Optional: Save out to format for Gephi Visualizer software

In [ ]:
# nx.set_node_attributes(songs_largest_component, songs_coms, "community")
# nx.set_node_attributes(songs_largest_component, songs_ud_degrees, "degrees")
# nx.set_node_attributes(songs_largest_component, song_partitions, "partitions")
# nx.write_gml(songs_largest_component, "songnet_lc.gml")

### Optional: Plot Communities

In [ ]:
#Image(filename = "SongNet_Communities.png")